Data preparation for Haddad OSA article


In [1]:
import pandas as pd
import numpy as np

Taxonomic feature table: Samples x Taxonomic species

In [3]:
taxa = pd.read_csv('data/taxonomic_observed_abundance_HaddadOSA.csv')
taxa = taxa.set_index('#SampleID')

In [4]:
taxa.head()

,14-2,1XD42-69,1XD8-76,Acetatifactor,Acetoanaerobium,Achromobacter,Acidovorax,Acinetobacter,Actinomycetospora,Acutalibacter,...,UBA7109,UBA8904,UBA964,UTPLA1,Unknown,UphvI-Ar2,Usitatibacter,Variovorax,Ventrimonas,XBB1006
#SampleID,,,,,,,,,,,,,,,,,,,,,
10422.17.F.10,9.0,3.0,1.0,15.0,0.0,0.0,0.0,0.0,0.0,2.0,...,1.0,0.0,0.0,0.0,1267.0,0.0,0.0,0.0,0.0,0.0
10422.17.F.11,15.0,2.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,32.0,...,10.0,0.0,0.0,0.0,1131.0,0.0,0.0,0.0,1.0,0.0
10422.17.F.12,16.0,4.0,2.0,11.0,0.0,0.0,0.0,0.0,0.0,17.0,...,12.0,0.0,0.0,0.0,1106.0,0.0,0.0,0.0,1.0,0.0
10422.17.F.13,38.0,13.0,3.0,10.0,0.0,0.0,0.0,0.0,0.0,22.0,...,3.0,0.0,0.0,0.0,938.0,0.0,0.0,0.0,0.0,0.0
10422.17.F.3,5.0,4.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,10.0,...,2.0,0.0,0.0,0.0,948.0,0.0,0.0,0.0,4.0,0.0


Meta-data:

In [9]:
metadata = pd.read_csv('original/haddad_6weeks_metadata_matched.txt', sep='\t')

Which metadata we care about: 
* age / collection_timestamp for time-series data. Or maybe better, use the Description to extract the serial number of the collection.
* IHH exposure - exposure_type - different between control and study-group. IHH - study_group, Air - Control
* host_subject_id / mouse_number


Some information about the experiment: 
Should be 16 mouse, 8 control. 
Each mouse should have 3 timestamps samples (feces).
 

In [10]:
metadata['exposure_type'].unique()

array(['IHH', 'Air'], dtype=object)

In [11]:
metadata['host_subject_id'].unique()

array(['Mouse 17', 'Mouse 18', 'Mouse 19', 'Mouse 20', 'Mouse 21',
       'Mouse 22', 'Mouse 23', 'Mouse 24', 'Mouse 25', 'Mouse 26',
       'Mouse 27', 'Mouse 28', 'Mouse 29', 'Mouse 30', 'Mouse 31',
       'Mouse 32'], dtype=object)

In [12]:
metadata = metadata.set_index('#SampleID')

In [13]:
metadata = pd.concat([metadata['host_subject_id'].to_frame()
                         , metadata['Description'].str.extract(r'.*collection (\d+) of .*').squeeze().to_frame('seq_sample_number'),
                      metadata['exposure_type'].map({'IHH': False, 'Air': True}).to_frame('control')], axis=1)

In [14]:
metadata['seq_sample_number'] = metadata['seq_sample_number'].astype(int)

Samples statistics from metadata:

In [15]:
# How manny control samples there are?
print(f'There are {metadata[metadata["control"]].shape[0]} samples under control (including multiple-timestamps per mice)')

There are 90 samples under control (including multiple-timestamps per mice)


In [16]:
print(f"There are {metadata.groupby('host_subject_id').apply(lambda x: x.sort_values(by='seq_sample_number', ascending=True).iloc[0]).query('control').shape[0]} under control, when taking only the first sample")

There are 8 under control, when taking only the first sample


In [18]:
metadata.to_csv('relevant_metadata_haddad_osa.csv')

Metabolites:

Load feature table of samples x metabolites (molecule annotations)

Note this data is after pqn normalization (what they did in the article) 
 probabilistic quotient normalization to an internal standard (m/z 278.189; real time, 3.81 min) for subsequent analysis: https://pubmed.ncbi.nlm.nih.gov/16808434/

Metabolite feature was annotated in OSA article using two methods: 
"standard identification" label as "level_1_identification", very high level, have 12 unique metabolite label with a lot of compounds mapped to the same label.
The "standard method", using theoretical matching to obs_mz/obs_rt.  
GNP label as "level_2_identification", more descriptive and fine-grained. GNPS method is algorithmic, data dependent method.  
We choose to work with GNP labels as the "compound" (name convention from Efrat's work on humans) in this article.  
 

Using GNPS we have 201 identified un-targeted metabolites, out of 1710 features of metabolites (aka most metabolites are not identified)

As many metabolite features can be mapped to the same GNPs we will group and sum those. 


In [63]:
metabolite_features = pd.read_table('original/haddad_6weeks_allFeatures_pqn_matched_wIDs.txt', sep ='\t')

In [64]:
metabolite_features.shape

(1710, 187)

In [65]:
metabolite_features.head()

,#featureID,10422.17.F.10,10422.17.F.11,10422.17.F.12,10422.17.F.13,10422.17.F.3,10422.17.F.4,10422.17.F.5,10422.17.F.6,10422.17.F.7,...,10422.32.F.4,10422.32.F.5,10422.32.F.6,10422.32.F.7,10422.32.F.8,10422.32.F.9,obs_mz,obs_rt,level2_gnps,level_1_identification
0,132.1020923761221_0.4491537660256411,4.178184e+08,5.991767e+08,2.710109e+08,3.743728e+08,3.945791e+08,2.378697e+08,6.067018e+08,1.485607e+08,4.227354e+08,...,2.788358e+08,5.916759e+08,5.721519e+08,4.399011e+08,3.049192e+08,7.210507e+08,132.102092,26.949226,none,none
1,166.0862755041856_0.5858793269230769,3.634780e+08,6.513032e+08,2.239334e+08,3.676121e+08,3.633605e+08,2.232058e+08,5.965530e+08,9.996564e+07,3.635239e+08,...,2.763446e+08,6.018018e+08,6.096148e+08,4.788034e+08,2.669838e+08,9.595680e+08,166.086276,35.152760,Massbank:PB006064 Phenylalanine|(2S)-2-amino-3...,none
2,357.2784138555112_5.010000161030595,3.730101e+08,4.353000e+08,4.358499e+08,3.920798e+08,4.798095e+08,6.040222e+08,4.506884e+08,3.380187e+08,3.516232e+08,...,4.220846e+08,3.300669e+08,2.797125e+08,3.749462e+08,2.102451e+08,4.921743e+08,357.278414,300.600010,Spectral Match to 3.beta.-Hydroxy-5-cholenoic ...,Deoxycholic acid
3,104.1070008277893_0.3359217147435899,5.302187e+08,5.266541e+08,2.206500e+08,4.904235e+08,3.866654e+08,2.319210e+08,4.159313e+08,3.514383e+08,4.247292e+08,...,3.700028e+08,4.422595e+08,4.189123e+08,4.235277e+08,2.993632e+08,5.883732e+08,104.107001,20.155303,none,none
4,373.2735011621581_3.9791876602564105,3.504783e+08,7.103335e+08,5.852700e+08,6.007156e+08,4.190507e+08,3.375733e+08,5.356349e+08,4.656738e+08,5.054181e+08,...,5.776752e+08,5.265270e+08,6.323558e+08,7.072081e+08,3.048172e+08,7.343837e+08,373.273501,238.751260,Spectral Match to Cholic acid from NIST14,b-Muricholic acid


In [67]:
gnp_metabolite = metabolite_features[metabolite_features['level2_gnps']!= 'none'].drop(columns=['#featureID','level_1_identification', 'obs_mz','obs_rt']).groupby('level2_gnps').sum()


In [68]:
gnp_metabolite.shape

(201, 182)

In [69]:
gnp_metabolite = gnp_metabolite.T

In [146]:
gnp_metabolite.head()

level2_gnps,()-Myristoylcarnitine,(S)-Equol,0498_Daidzein,"1,2,3,4-tetrahydroharmane-3-carboxylic acid",1-(9Z-Octadecenoyl)-sn-glycero-3-phosphocholine,1-Stearoyl-2-hydroxy-sn-glycero-3-phosphocholine,1-Stearoyl-2-hydroxy-sn-glycero-3-phosphoethanolamine,11Z-hexadecenoic acid,"13S-Hydroxy-9Z,11E,15Z-octadecatrienoic acid",1OPalmitoyl2Oacetylsnglycero3phosphorylcholine,...,Ursodiol,Val Ile Ser,Val Ile Thr,Val Leu Ile,Val Pro Val,Val Tyr,Val Val Leu,cis-7-Hexadecenoic acid methyl ester,none_16904_dereplictor_pv_3.86203e-22,sucrose
10422.17.F.10,3.159649e+08,3.922909e+08,4.118110e+08,1.424319e+08,5.126184e+08,8.969007e+07,4.198768e+08,5.648033e+07,1.184549e+09,1.108540e+09,...,5.528667e+08,2.181808e+08,3.471831e+08,1.534835e+08,1.821822e+08,1.407291e+08,2.693802e+08,1.256537e+08,0.000000e+00,1.000284e+09
10422.17.F.11,2.126023e+07,1.867059e+08,0.000000e+00,4.496135e+08,1.727695e+08,2.019812e+08,3.141754e+07,1.046284e+08,1.934521e+09,2.901169e+08,...,1.452149e+09,9.307885e+08,3.394768e+09,8.419900e+08,6.658940e+08,6.349237e+08,1.361782e+09,2.059950e+08,3.319443e+08,1.110240e+09
10422.17.F.12,2.336577e+08,3.954004e+08,2.159655e+08,1.159784e+08,2.714844e+08,2.939630e+08,3.047576e+08,1.251146e+08,1.904190e+09,1.127209e+08,...,9.390147e+08,3.715455e+08,1.247775e+09,4.383393e+08,3.722912e+07,1.393732e+08,3.297086e+10,2.709687e+08,6.959110e+08,2.403973e+09
10422.17.F.13,2.502001e+08,8.583749e+07,3.043807e+08,7.896079e+08,6.983418e+08,6.601377e+08,5.279404e+08,2.304841e+08,2.999902e+09,5.924575e+09,...,1.072192e+09,1.465024e+09,2.544299e+09,8.091601e+08,1.091217e+09,1.016178e+09,1.253986e+09,5.420607e+08,3.120713e+09,7.512926e+09
10422.17.F.3,1.676555e+09,1.524704e+07,4.848800e+08,1.208026e+09,1.179622e+09,2.965064e+09,2.442140e+09,1.394876e+08,1.313219e+09,4.320742e+08,...,3.081732e+08,9.684140e+08,1.597777e+09,4.121136e+08,5.428282e+08,4.873222e+08,6.365133e+08,4.903278e+08,4.637753e+07,4.109085e+08


In [147]:
gnp_metabolite.to_csv('data/metabolite_gnp_annotated_HaddadOSA.csv')

Map metabolites GNPS annotation to HMDB and KEGG ids:

In [132]:
metabolite_features = pd.read_csv('data/metabolite_gnp_annotated_HaddadOSA.csv', index_col='Unnamed: 0')

In [133]:
metabolite_features = metabolite_features.stack().reset_index(drop=False).rename(columns={'level_0': 'sample_id', 'level_1':'metabolite_id' ,0:'metabolite_level'})

In [134]:
metabolite_features.head()

,sample_id,metabolite_id,metabolite_level
0,10422.17.F.10,()-Myristoylcarnitine,3.159649e+08
1,10422.17.F.10,(S)-Equol,3.922909e+08
2,10422.17.F.10,0498_Daidzein,4.118110e+08
3,10422.17.F.10,"1,2,3,4-tetrahydroharmane-3-carboxylic acid",1.424319e+08
4,10422.17.F.10,1-(9Z-Octadecenoyl)-sn-glycero-3-phosphocholine,5.126184e+08


Map the GNPS metabolites to HMDB

In [135]:
gnps_indexes = pd.read_csv('data/gnps_indexes_to_hmdb_ids.csv')
gnps_to_hmdb_ids = gnps_indexes.set_index('GNPS indexes')['HMDB id']


In [106]:
gnps_to_hmdb_ids.shape

(121,)

In [143]:
print(f"There are {gnps_to_hmdb_ids.dropna().shape[0]} metabolites mapped from GNPS to HMDB, {gnps_to_hmdb_ids.dropna().unique().shape[0]} unique HMDB ids, out of {gnps_to_hmdb_ids.shape[0]} GNPS ids")

There are 73 metabolites mapped from GNPS to HMDB, 71 unique HMDB ids, out of 121 GNPS ids


In [138]:
metabolite_features['metabolite_id'] = metabolite_features['metabolite_id'].map(gnps_to_hmdb_ids)

In [140]:
metabolite_features = metabolite_features.pivot_table(columns='metabolite_id', aggfunc='sum', index='sample_id', values='metabolite_level')

In [148]:
metabolite_features.to_csv('data/metabolite_HMDB_annotated_HaddadOSA.csv')